# 2.2 ClimateWins Keras- Hyperparameter Optimization with Bayesian Search

## Contents:

### 1. Importing Libraries and Data
- Libraries in line with Keras, 2.2 scripts
- Data: unscaled (observations), pleasant weather (predictions)

### 2. Data Wranging

### 3. Reshaping for ML modeling and Bayesian Optomization
the Bayesian optimization requires a slightly different form for your X and y sets. Before you split them:

-Your X shape will need to be in the form (22950, 15, 9);

-Your y shape will need to be in the form (22950,);

You can use the argmax function shown in the Exercise to transform your y set.

### 4. Split data

## 5. Bayesian Hyperparameter Optimzation

### 6. Create Keras Model


- Compile and Run the model

- Create confusion matrix of results
    

In [40]:
# Importing libraries for Keras, CNNs, and Hyperparameter Optimization

import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scikeras.wrappers import KerasClassifier

from sklearn.model_selection import train_test_split, cross_val_score
import tensorflow as tf
from sklearn import datasets  
from sklearn.ensemble import RandomForestClassifier
from numpy import argmax
from sklearn import metrics  
from sklearn.tree import plot_tree
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings

/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [3]:
print (keras.__version__)

3.3.3


In [4]:
# Setting Path 

path = r'/Users/jeremyobach/Documents/Data Analytics/CareerFoundry/Specialization - Machine Learning/Real World Applications of Machine Learning/ML Achievement 2 MASTER FOLDER'
path

'/Users/jeremyobach/Documents/Data Analytics/CareerFoundry/Specialization - Machine Learning/Real World Applications of Machine Learning/ML Achievement 2 MASTER FOLDER'

In [5]:
# delimiting columns displayed
pd.options.display.max_columns = None

In [6]:
# Importing prediction data

pleasant = pd.read_csv (os.path.join(path, '02 Data', 'pleasant_wx_predictions.csv'), index_col = False)

In [7]:
# Importing unscaled observation data

unscaled =  pd.read_csv (os.path.join(path, '02 Data', 'wx_unscaled.csv'), index_col = False)

In [7]:
unscaled.shape

(22950, 170)

In [8]:
pleasant.shape

(22950, 16)

### 2. Data Wrangling

- Ensure wx data is structured with correct shape to feed the deep learning model.
    - drop DATE and MONTH from observations, DATE from predictions
    - drop 3 wx stations not included in pleasant_wx
    - 2 types of observations (columns) are missing multiple years for most wx stations; remove them
    - 3 individual observations need to be filled in
    - Export data as "Cleaned" version, X shape should be (22950, 135) and y shape should be (22950, 15)

In [8]:
# dropping all columns regarding Gdansk, Roma, Tours from unscaled, as they aren't included in pleasant_wx


columns_to_drop = ['GDANSK_cloud_cover', 'GDANSK_humidity', 'GDANSK_precipitation', 
                   'GDANSK_snow_depth', 'GDANSK_temp_mean', 'GDANSK_temp_min', 
                   'GDANSK_temp_max', 'ROMA_cloud_cover', 'ROMA_wind_speed', 
                   'ROMA_humidity', 'ROMA_pressure', 'ROMA_sunshine', 'ROMA_temp_mean',
                   'TOURS_wind_speed', 'TOURS_humidity', 'TOURS_pressure',
                   'TOURS_global_radiation', 'TOURS_precipitation', 'TOURS_temp_mean', 
                   'TOURS_temp_min', 'TOURS_temp_max']

# Using the drop() method to drop the specified columns
unscaled.drop(columns=columns_to_drop, inplace=True)

In [9]:
unscaled.shape 

(22950, 149)

In [10]:
unscaled.drop(columns = ['DATE', 'MONTH'], inplace = True)

In [11]:
pleasant.drop(columns = 'DATE', inplace = True) 

In [12]:
pleasant.shape

# prediction dataset is correct shape

(22950, 15)

Two types of observations are missing multiple years for most weather stations. Remove them.

- wind speed (only 11 stations)
- snow depth (only 7 stations)

In [13]:
further_drops = ['BASEL_snow_depth',  'DUSSELDORF_snow_depth', 'HEATHROW_snow_depth',
                  'MUNCHENB_snow_depth', 'OSLO_snow_depth',  'VALENTIA_snow_depth',
                 'BASEL_wind_speed', 'DEBILT_wind_speed', 'DUSSELDORF_wind_speed',
                  'KASSEL_wind_speed', 'LJUBLJANA_wind_speed',  'MAASTRICHT_wind_speed',
                  'MADRID_wind_speed', 'OSLO_wind_speed','SONNBLICK_wind_speed',]

unscaled.drop(columns=further_drops, inplace=True)

In [14]:
# Inserting "dummy values" for the 3 that are still missing

# missing 'Kassel_cloud_cover', with 'Ljubjana_cloud_cover'
# missing 'Munchenb_pressure' with 'Sonnblick_pressure'
# missing 'Stockholm_humidity' with Oslo_humidity'

unscaled.columns.get_loc('HEATHROW_temp_max')

53

In [15]:
unscaled.columns.get_loc('MUNCHENB_humidity')

90

In [16]:
unscaled.columns.get_loc('STOCKHOLM_cloud_cover')

115

In [17]:
unscaled.insert(54,'KASSEL_cloud_cover', unscaled['LJUBLJANA_cloud_cover'])
unscaled.insert(92,'MUNCHENB_pressure',unscaled['SONNBLICK_pressure'])
unscaled.insert(118, 'STOCKHOLM_humidity', unscaled['OSLO_humidity'])
unscaled.columns.tolist()

['BASEL_cloud_cover',
 'BASEL_humidity',
 'BASEL_pressure',
 'BASEL_global_radiation',
 'BASEL_precipitation',
 'BASEL_sunshine',
 'BASEL_temp_mean',
 'BASEL_temp_min',
 'BASEL_temp_max',
 'BELGRADE_cloud_cover',
 'BELGRADE_humidity',
 'BELGRADE_pressure',
 'BELGRADE_global_radiation',
 'BELGRADE_precipitation',
 'BELGRADE_sunshine',
 'BELGRADE_temp_mean',
 'BELGRADE_temp_min',
 'BELGRADE_temp_max',
 'BUDAPEST_cloud_cover',
 'BUDAPEST_humidity',
 'BUDAPEST_pressure',
 'BUDAPEST_global_radiation',
 'BUDAPEST_precipitation',
 'BUDAPEST_sunshine',
 'BUDAPEST_temp_mean',
 'BUDAPEST_temp_min',
 'BUDAPEST_temp_max',
 'DEBILT_cloud_cover',
 'DEBILT_humidity',
 'DEBILT_pressure',
 'DEBILT_global_radiation',
 'DEBILT_precipitation',
 'DEBILT_sunshine',
 'DEBILT_temp_mean',
 'DEBILT_temp_min',
 'DEBILT_temp_max',
 'DUSSELDORF_cloud_cover',
 'DUSSELDORF_humidity',
 'DUSSELDORF_pressure',
 'DUSSELDORF_global_radiation',
 'DUSSELDORF_precipitation',
 'DUSSELDORF_sunshine',
 'DUSSELDORF_temp_mean',


In [18]:
unscaled.shape

(22950, 135)

In [19]:
unscaled.to_pickle(os.path.join(path, '02 Data', 'X_cleaned.pkl'))

### 3. Reshaping for ML modeling
- Ensure the layers can be fed to the deep learning model correctly. 
- You’ll need to split the observations (X) into 15 groups of 9 types of observations, and your labels (y) should also be in 15 groups 
    - (it doesn’t need to be transformed or reshaped). 
        - The final shapes should be X = (22950, 15, 9) and y = (22950, 15).


In [20]:
X = pd.read_pickle(os.path.join(path, '02 Data', 'X_cleaned.pkl'))

In [25]:
y = pleasant

In [26]:
X.shape

(22950, 15, 9)

In [27]:
# turning X and y from a df to arrays

X = np.array(X)
y = np.array(y)

In [28]:
X = X.reshape(-1,15,9)

In [29]:
y =  np.argmax(y, axis = 1)

In [30]:
# verifying array shape

y

array([0, 0, 0, ..., 0, 0, 0])

In [31]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

### 4. Splitting data (training and test sets)

In [32]:
#Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 39)

## 5. Bayesian hyperparameter optimization

In [42]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 #_count_classes(Y_train) #Number of weather stations!!
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [43]:
# Create function
def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]
    optimizer = optimizerL[round(optimizer)] #optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [44]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), #9
    'optimizer':(0,7), #7
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), #(10, 50), #
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) #25
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/32


/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 - 1s - 96ms/step - accuracy: 0.5999 - loss: 2.7083
Epoch 2/32
15/15 - 1s - 51ms/step - accuracy: 0.6425 - loss: 2.7004
Epoch 3/32
15/15 - 1s - 51ms/step - accuracy: 0.6425 - loss: 2.6971
Epoch 4/32
15/15 - 1s - 50ms/step - accuracy: 0.6425 - loss: 2.6942
Epoch 5/32
15/15 - 1s - 51ms/step - accuracy: 0.6425 - loss: 2.6917
Epoch 6/32
15/15 - 1s - 51ms/step - accuracy: 0.6425 - loss: 2.6894
Epoch 7/32
15/15 - 1s - 51ms/step - accuracy: 0.6425 - loss: 2.6873
Epoch 8/32
15/15 - 1s - 51ms/step - accuracy: 0.6425 - loss: 2.6854
Epoch 9/32
15/15 - 1s - 56ms/step - accuracy: 0.6425 - loss: 2.6835
Epoch 10/32
15/15 - 1s - 52ms/step - accuracy: 0.6425 - loss: 2.6818
Epoch 11/32
15/15 - 1s - 51ms/step - accuracy: 0.6425 - loss: 2.6801
Epoch 12/32
15/15 - 1s - 52ms/step - accuracy: 0.6425 - loss: 2.6785
Epoch 13/32
15/15 - 1s - 52ms/step - accuracy: 0.6425 - loss: 2.6769
Epoch 14/32
15/15 - 1s - 52ms/step - accuracy: 0.6425 - loss: 2.6754
Epoch 15/32
15/15 - 1s - 52ms/step - accuracy: 0.6425 

/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 - 1s - 91ms/step - accuracy: 0.5990 - loss: 2.7046
Epoch 2/32
15/15 - 1s - 52ms/step - accuracy: 0.6425 - loss: 2.7004
Epoch 3/32
15/15 - 1s - 54ms/step - accuracy: 0.6425 - loss: 2.6971
Epoch 4/32
15/15 - 1s - 52ms/step - accuracy: 0.6425 - loss: 2.6942
Epoch 5/32
15/15 - 1s - 63ms/step - accuracy: 0.6425 - loss: 2.6917
Epoch 6/32
15/15 - 1s - 52ms/step - accuracy: 0.6425 - loss: 2.6894
Epoch 7/32
15/15 - 1s - 51ms/step - accuracy: 0.6425 - loss: 2.6873
Epoch 8/32
15/15 - 1s - 52ms/step - accuracy: 0.6425 - loss: 2.6853
Epoch 9/32
15/15 - 1s - 52ms/step - accuracy: 0.6425 - loss: 2.6834
Epoch 10/32
15/15 - 1s - 51ms/step - accuracy: 0.6425 - loss: 2.6817
Epoch 11/32
15/15 - 1s - 51ms/step - accuracy: 0.6425 - loss: 2.6800
Epoch 12/32
15/15 - 1s - 50ms/step - accuracy: 0.6425 - loss: 2.6783
Epoch 13/32
15/15 - 1s - 51ms/step - accuracy: 0.6425 - loss: 2.6768
Epoch 14/32
15/15 - 1s - 54ms/step - accuracy: 0.6425 - loss: 2.6752
Epoch 15/32
15/15 - 1s - 54ms/step - accuracy: 0.6425 

...

/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 - 1s - 41ms/step - accuracy: 0.5088 - loss: 1.6114
Epoch 2/61
21/21 - 0s - 15ms/step - accuracy: 0.6616 - loss: 1.0143
Epoch 3/61
21/21 - 0s - 15ms/step - accuracy: 0.6909 - loss: 0.9007
Epoch 4/61
21/21 - 0s - 15ms/step - accuracy: 0.7135 - loss: 0.8413
Epoch 5/61
21/21 - 0s - 14ms/step - accuracy: 0.7272 - loss: 0.7975
Epoch 6/61
21/21 - 0s - 14ms/step - accuracy: 0.7354 - loss: 0.7650
Epoch 7/61
21/21 - 0s - 14ms/step - accuracy: 0.7455 - loss: 0.7341
Epoch 8/61
21/21 - 0s - 14ms/step - accuracy: 0.7534 - loss: 0.7089
Epoch 9/61
21/21 - 0s - 14ms/step - accuracy: 0.7646 - loss: 0.6764
Epoch 10/61
21/21 - 0s - 14ms/step - accuracy: 0.7755 - loss: 0.6467
Epoch 11/61
21/21 - 0s - 14ms/step - accuracy: 0.7887 - loss: 0.6132
Epoch 12/61
21/21 - 0s - 15ms/step - accuracy: 0.7952 - loss: 0.5848
Epoch 13/61
21/21 - 0s - 14ms/step - accuracy: 0.8054 - loss: 0.5536
Epoch 14/61
21/21 - 0s - 14ms/step - accuracy: 0.8161 - loss: 0.5206
Epoch 15/61
21/21 - 0s - 14ms/step - accuracy: 0.8293 

/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/jeremyobach/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 - 1s - 41ms/step - accuracy: 0.5665 - loss: 1.4567
Epoch 2/61
21/21 - 0s - 15ms/step - accuracy: 0.6565 - loss: 0.9801
Epoch 3/61
21/21 - 0s - 15ms/step - accuracy: 0.6841 - loss: 0.8973
Epoch 4/61
21/21 - 0s - 15ms/step - accuracy: 0.7117 - loss: 0.8317
Epoch 5/61
21/21 - 0s - 15ms/step - accuracy: 0.7284 - loss: 0.7822
Epoch 6/61
21/21 - 0s - 15ms/step - accuracy: 0.7381 - loss: 0.7419
Epoch 7/61
21/21 - 0s - 15ms/step - accuracy: 0.7516 - loss: 0.7033
Epoch 8/61
21/21 - 0s - 15ms/step - accuracy: 0.7635 - loss: 0.6695
Epoch 9/61
21/21 - 0s - 15ms/step - accuracy: 0.7716 - loss: 0.6430
Epoch 10/61
21/21 - 0s - 15ms/step - accuracy: 0.7853 - loss: 0.6103
Epoch 11/61
21/21 - 0s - 15ms/step - accuracy: 0.7920 - loss: 0.5851
Epoch 12/61
21/21 - 0s - 15ms/step - accuracy: 0.8015 - loss: 0.5579
Epoch 13/61
21/21 - 0s - 15ms/step - accuracy: 0.8116 - loss: 0.5296
Epoch 14/61
21/21 - 0s - 15ms/step - accuracy: 0.8206 - loss: 0.5063
Epoch 15/61
21/21 - 0s - 15ms/step - accuracy: 0.8277 

ValueError: Input y contains NaN.

In [46]:
best_params = nn_opt.max['params']
best_score = nn_opt.max['target']

print(f"Best Parameters: {best_params}")
print(f"Highest Accuracy: {best_score}")

Best Parameters: {'activation': 2.79884089544096, 'batch_size': 460.14665762139765, 'dropout': 0.7296061783380641, 'dropout_rate': 0.19126724140656393, 'epochs': 90.97701940610612, 'kernel': 1.9444298503238986, 'layers1': 1.2391884918766034, 'layers2': 2.42648957444599, 'learning_rate': 0.7631771981307285, 'neurons': 60.51494778125466, 'normalization': 0.770967179954561, 'optimizer': 3.456569174550735}
Highest Accuracy: nan


In [47]:
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', 'LeakyReLU', 'relu']
optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'SGD']

# Best parameters from optimization
activation_index = round(best_params['activation'])
optimizer_index = round(best_params['optimizer'])

best_params['activation'] = activationL[activation_index]
best_params['optimizer'] = optimizerL[optimizer_index]
best_params['neurons'] = round(best_params['neurons'])
best_params['kernel'] = round(best_params['kernel'])
best_params['batch_size'] = round(best_params['batch_size'])
best_params['epochs'] = round(best_params['epochs'])
best_params['layers1'] = round(best_params['layers1'])
best_params['layers2'] = round(best_params['layers2'])

print(f"Best Parameters (interpreted): {best_params}")


Best Parameters (interpreted): {'activation': 'softsign', 'batch_size': 460, 'dropout': 0.7296061783380641, 'dropout_rate': 0.19126724140656393, 'epochs': 91, 'kernel': 2, 'layers1': 1, 'layers2': 2, 'learning_rate': 0.7631771981307285, 'neurons': 61, 'normalization': 0.770967179954561, 'optimizer': 'Adadelta'}


### 6. Keras Model

In [90]:
# Best parameters from optimization
best_params = {
    'neurons': 61,
    'kernel': 2,
    'activation': 'softsign',
    'optimizer': 'Adadelta',
    'learning_rate': 0.76317719813,
    'batch_size': 460,
    'epochs': 91,
    'layers1': 1,
    'layers2': 2,
    'normalization': 0.770967179954561,
    'dropout': 0.7296061783380641,
    'dropout_rate':  0.19126724140656393
}

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl

# Initialize optimizer with learning rate
optimizers = {
    'Adam': Adam(learning_rate=best_params['learning_rate']),
    'SGD': SGD(learning_rate=best_params['learning_rate']),
    'RMSprop': RMSprop(learning_rate=best_params['learning_rate']),
    'Adadelta': Adadelta(learning_rate=best_params['learning_rate']),
    'Adagrad': Adagrad(learning_rate=best_params['learning_rate']),
    'Adamax': Adamax(learning_rate=best_params['learning_rate']),
    'Nadam': Nadam(learning_rate=best_params['learning_rate']),
    'Ftrl': Ftrl(learning_rate=best_params['learning_rate'])
}

optimizer = optimizers[best_params['optimizer']]

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
#n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(best_params['neurons'], kernel_size=best_params['kernel'], activation=best_params['activation'], input_shape=(15,9)))

if best_params['normalization'] > 0.5:
    model.add(BatchNormalization())

for _ in range(best_params['layers1']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

if best_params['dropout'] > 0.5:
    model.add(Dropout(best_params['dropout_rate'], seed=123))

for _ in range(best_params['layers2']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # sigmoid, tanh, softmax

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model with the optimized parameters
model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=2)


Epoch 1/91
38/38 - 1s - 31ms/step - accuracy: 0.6005 - loss: 1.4012
Epoch 2/91
38/38 - 1s - 15ms/step - accuracy: 0.6871 - loss: 0.9071
Epoch 3/91
38/38 - 1s - 16ms/step - accuracy: 0.7140 - loss: 0.8166
Epoch 4/91
38/38 - 1s - 16ms/step - accuracy: 0.7363 - loss: 0.7568
Epoch 5/91
38/38 - 1s - 17ms/step - accuracy: 0.7542 - loss: 0.7112
Epoch 6/91
38/38 - 1s - 16ms/step - accuracy: 0.7671 - loss: 0.6761
Epoch 7/91
38/38 - 1s - 17ms/step - accuracy: 0.7782 - loss: 0.6411
Epoch 8/91
38/38 - 1s - 16ms/step - accuracy: 0.7892 - loss: 0.6095
Epoch 9/91
38/38 - 1s - 16ms/step - accuracy: 0.7996 - loss: 0.5801
Epoch 10/91
38/38 - 1s - 16ms/step - accuracy: 0.8090 - loss: 0.5560
Epoch 11/91
38/38 - 1s - 16ms/step - accuracy: 0.8123 - loss: 0.5397
Epoch 12/91
38/38 - 1s - 16ms/step - accuracy: 0.8169 - loss: 0.5203
Epoch 13/91
38/38 - 1s - 16ms/step - accuracy: 0.8298 - loss: 0.4931
Epoch 14/91
38/38 - 1s - 15ms/step - accuracy: 0.8327 - loss: 0.4857
Epoch 15/91
38/38 - 1s - 15ms/step - accura

In [51]:
model.summary()

Model: "sequential_85"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_85 (Conv1D)              │ (None, 14, 61)         │         1,159 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_30          │ (None, 14, 61)         │           244 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_410 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_50 (Dropout)            │ (None, 14, 61)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_411 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_412 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_80 (MaxPooling1D) │ (None, 7, 61)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_80 (Flatten)            │ (None, 427)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_413 (Dense)               │ (None, 15)             │         6,420 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 57,265 (223.70 KB)

 Trainable params: 19,047 (74.40 KB)

 Non-trainable params: 122 (488.00 B)

 Optimizer params: 38,096 (148.82 KB)

In [91]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [92]:
from tensorflow.keras.utils import to_categorical


# Assuming X_train and y_train are your input data and labels
# One-hot encode y_train
y_train_one_hot = to_categorical(y_train, num_classes=15)

In [93]:
# Check shapes
print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [94]:
model.fit(X_train, y_train_one_hot, batch_size=460, epochs=91, verbose=2)

Epoch 1/91
38/38 - 1s - 32ms/step - accuracy: 0.9490 - loss: 0.1440
Epoch 2/91
38/38 - 1s - 16ms/step - accuracy: 0.9550 - loss: 0.1284
Epoch 3/91
38/38 - 1s - 15ms/step - accuracy: 0.9547 - loss: 0.1338
Epoch 4/91
38/38 - 1s - 16ms/step - accuracy: 0.9573 - loss: 0.1278
Epoch 5/91
38/38 - 1s - 16ms/step - accuracy: 0.9582 - loss: 0.1195
Epoch 6/91
38/38 - 1s - 16ms/step - accuracy: 0.9603 - loss: 0.1126
Epoch 7/91
38/38 - 1s - 17ms/step - accuracy: 0.9606 - loss: 0.1156
Epoch 8/91
38/38 - 1s - 16ms/step - accuracy: 0.9560 - loss: 0.1289
Epoch 9/91
38/38 - 1s - 19ms/step - accuracy: 0.9625 - loss: 0.1106
Epoch 10/91
38/38 - 1s - 16ms/step - accuracy: 0.9606 - loss: 0.1160
Epoch 11/91
38/38 - 1s - 16ms/step - accuracy: 0.9640 - loss: 0.1096
Epoch 12/91
38/38 - 1s - 16ms/step - accuracy: 0.9625 - loss: 0.1117
Epoch 13/91
38/38 - 1s - 16ms/step - accuracy: 0.9628 - loss: 0.1060
Epoch 14/91
38/38 - 1s - 15ms/step - accuracy: 0.9604 - loss: 0.1128
Epoch 15/91
38/38 - 1s - 16ms/step - accura

In [160]:
#Change this to Weather true/false
stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [161]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])



In [162]:
y_pred = model.predict(X_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 614us/step


In [163]:
cm = confusion_matrix(y_test, y_pred, stations)
print(cm)

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3663        27         4       2           1         1       0   
BELGRADE       49      1029         1       0           0         0       0   
BUDAPEST       11         7       174       3           0         1       0   
DEBILT          1         4         7      74           1         2       0   
DUSSELDORF      4         2         0       1          19         5       0   
HEATHROW        4         1         1       1           3        95       0   
KASSEL          0         1         0       1           1         0      10   
LJUBLJANA       6         1         0       0           0         0       1   
MAASTRICHT      0         0         0       1           0         0       0   
MADRID         19         7         3       0           0         4       0   
MUNCHENB        1         0         0       0       